In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
from glob import glob
path = '../input/v2-plant-seedlings-dataset'
class_names = []
class_label = []
images = []
classes = []

image_per_class = {}
for class_folder in os.listdir(path):
    
    
    class_folder_path = os.path.join(path, class_folder)
    print(class_folder_path)
    class_label = class_folder
    class_names.append(class_folder)
    image_per_class[class_label] = []
    for images_path in glob(os.path.join(class_folder_path, "*.png")):
        image_bgr = cv2.imread(images_path, cv2.IMREAD_COLOR)
        image_per_class[class_label].append(image_bgr)
        images.append(image_bgr)
        classes.append(class_label)
        
        

        
        




In [ ]:
print(type(image_per_class))

for key, values in image_per_class.items():
    print("{0}--->{1}".format(key, len(values)))


In [ ]:
images = np.array(images)
print(type(images))

In [ ]:
print(len(class_names))
print(len(classes))
print(len(images))
#size of image
print( (images[1].shape))
print(classes[1030:1036])

In [ ]:
scale = 256
def resize_image(img):
    img = np.array(img).astype(np.uint8)
    imag = cv2.resize(img, (scale,scale), interpolation = cv2.INTER_CUBIC)
    #print(type(imag))
    return imag



In [ ]:
# image resize making compatible for vgg16() model
images = [resize_image(img) for img in images]
images = np.array(images)
print(type(images))

In [ ]:
#before giving images to model they are segmented for better result



def mask_image(image):
    image = np.array(image).astype(np.uint8)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask


def segment_image(image):
    mask = mask_image(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output


def sharpen_image(image):
    blurred_image = cv2.GaussianBlur(image, (0, 0), 3)
    sharp_image = cv2.addWeighted(image, 1.5, blurred_image, -0.5, 0)
    return sharp_image


def process_images(img):    
    segmented_image = segment_image(img)
    sharpened_image = sharpen_image(segmented_image)
    return sharpened_image

In [ ]:
#processing all images for better result/accuracy
print(type(images))
for index, image in enumerate(images):
    images[index] = process_images(image)
    
    
    

In [ ]:
#import matplotlib.pyplot as plt
#image = images[classes == "Common Chickweed"][2]
#plt.imshow(image)
#print(type(image))


In [ ]:
#mask_image_ = mask_image(image)
#segmented_image = segment_image(image)
#sharpened_image = sharpen_image(segmented_image)

#fig, axs = plt.subplots(1, 4, figsize=(20, 20))
#axs[0].imshow(image)
#axs[1].imshow(mask_image_)
#axs[2].imshow(segmented_image)
#axs[3].imshow(sharpened_image)

In [ ]:
classes = np.array(classes)
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y = encoder.fit_transform(classes) #one hot encoded

#print(y[0])
#print(classes[0])
#print(class_names)
#print(len(class_names))


In [ ]:
#splitting dataset into train and test 
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(images, y, test_size = 0.2, random_state = 50)

print(X_train.shape)

In [ ]:
#CNN Model

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam



batch_size = None

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu',
                 batch_input_shape = (batch_size, 256, 256, 3) ))
model.add(Conv2D(filters = 64, kernel_size = (5, 5), padding = "Same", activation = 'relu') )
model.add(MaxPool2D(pool_size = (2, 2)) )
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "Same", activation = 'relu') )
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'Same', activation = 'relu') )          
model.add(MaxPool2D(pool_size=(2,2), strides = (2, 2)  ))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = "Same", activation = 'relu') )
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'Same', activation = 'relu') )          
model.add(MaxPool2D(pool_size = (2, 2), strides = (2, 2) ) )
model.add(Dropout(0.4))

model.add(GlobalMaxPooling2D())
model.add(Dense(256, activation = 'relu') )
model.add(Dropout(0.5))
model.add(Dense(12, activation = "softmax") )
model.summary()


In [ ]:
opt = Adam(lr = 0.001)
optimizer = RMSprop(lr = 0.001, rho= 0.9, epsilon = 1e-08, decay = 0.0 )
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))

In [ ]:
#print(len(Y_train[0]))
#for i in range(4431):
#    print(X_train[i].shape)

X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size = 0.5, random_state = 20)



In [ ]:
model.fit(X_train, Y_train, epochs = 10, validation_data = (X_val, Y_val), batch_size = batch_size)


In [ ]:
#model evaluation on validation data
score = model.evaluate(X_val, Y_val, verbose = 0.0, batch_size = batch_size)


In [ ]:
print("Validation Loss: {}".format(score[0]))
print("validdation accuracy: {}".format(score[1]))

In [ ]:
# data augmentation

aug = ImageDataGenerator(rotation_range = 10,
                         zoom_range = 0.1,
                         horizontal_flip = True,
                         vertical_flip = False,
                        width_shift_range = 0.1,
                        height_shift_range = 0.1)
#other parameters
#zca_whitening = True
#fill_mode = nearest

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor = "acc",
                                            patience =3,
                                            verbose = 1.0,
                                            factor = 0.5, 
                                           min_lr = 0.0001)

In [ ]:
trained_model = model.fit_generator(aug.flow(X_train, Y_train, batch_size = 38),
                                    epochs = 10,
                                    validation_data = (X_val,Y_val),
                                    verbose = 2, 
                                    callbacks = [learning_rate_reduction])

In [ ]:
score_aug = model.evaluate(X_val, Y_val, verbose = 2, batch_size = None, )
print("loss:{}".format(score_aug[0]))
print("Accuracy:{}".format(score_aug[1]))

In [ ]:
#plotting loss and accuracy
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 1, figsize = (15, 8))
ax[0].plot(trained_model.history["loss"], color = "r", label = "training_loss")
ax[0].plot(trained_model.history["val_loss"], color = "b", label = "validation_loss") 
legend = ax[0].legend(loc = 'best', shadow = True)

ax[1].plot(trained_model.history["accuracy"], color = "r", label = "training_accuracy")
ax[1].plot(trained_model.history["val_accuracy"], color = "b", label = "validation_accuracy")
legend = ax[1].legend(loc = 'best', shadow = True)


In [ ]:
predicted_class = model.predict_classes(X_test)
print(type(predicted_class))
print(predicted_class.shape)
wrong_prediction = X_test[predicted_class != np.argmax(Y_test)]
print(wrong_prediction.shape)
print(set(predicted_class))

In [ ]:
i=20
pred = model.predict_classes(np.array([wrong_prediction[i]]))[0]
act = np.argmax(Y_test[i])
print("Predicted class: {}".format(encoder.classes_[pred]))
print("Actual class: {}".format(encoder.classes_[act]))

plt.imshow(wrong_prediction[i])